In [1]:
import pandas as pd
import datetime
import re
import numpy as np
import datetime
import plotly.express as px
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import shutil
import sys
import os.path
import plotly.figure_factory as ff
import random

In [2]:
kuka_output = pd.read_excel (r'KukaOutput-ICNAP.xlsx')
oven_programs = pd.read_excel (r'OvenPrograms-ICNAP.xlsx')
oven_programs = oven_programs.rename(columns={'product type': 'Product', 
                                              'program name': 'Program', 
                                              'target temp (C)':'TargetTemp', 
                                              'program duration (min)':'ProgramDuration', 
                                              'cooling temp  (C)':'CoolingTemp', 
                                              'time to cooling (min)':'TimeCooling'})

data = pd.merge(kuka_output, 
                      oven_programs, 
                      on ='Product', 
                      how ='inner')
data = data.sort_values(by='EndDate', ascending=True)

input_stack = data[['EndDate','Product']]
input_stack.Product = input_stack.Product.map(lambda x: x.lstrip('Rotor '))

In [7]:
#(full/empty , high/low)
oven_status = np.array([[False,False], [False,False], [False,False], [False,False], [False,False], [False,False]])

#append tuple (Time In, Pallet No., Duration)
oven_stack = [[],[],[],[],[],[]]

oven_time = {'A0':(320,43),
       'A1':(320,43),
       'A2':(450,43),
       'A3':(450,43),
       'A4':(460,43),
       'A5':(460,43),
       'A6':(460,43),
       'B2':(300,71),
       'B3':(300,71),
       'B4':(450,71),
       'B5':(450,71),
       'B6':(450,71),
       'C0':(240,43),
       'C1':(240,43),
       'C2':(270,43),
       'C3':(270,43),
       'C4':(300,43),
       'C5':(300,43),
       'C6':(300,43)}
ovens = np.array([])
prodlist = ['A0','A1','A2','A3','A4','A5','A6','B1','B2','B3','B4','B5','B6','C0','C1','C2','C3','C4','C5','C6']
ovencounter = np.zeros((6,2))
choose = 0
busy = 0
for index, row in input_stack.iterrows():
    time_stamp = row.EndDate
    pallet_no = row.Product
    indd = prodlist.index(pallet_no)

    flag = 0
    (duration,cooling_temp) = oven_time[pallet_no]
    old_duration = 0
    old_time_stamp = datetime.datetime(2022,2,1)
    
    
    if ovens.shape[0] == 0:
        ovens = np.arange(6)
    
    for i in range(len(ovens)):
        
        choose = np.random.choice(ovens)
        if indd == ovencounter[choose,0] and ovencounter[choose,1] > 0:# and ovencounter[choose,1] < 4:
            oven_stack[choose].pop(-1)
            c = ovencounter[choose,1] 
            ovencounter[choose] = [indd,c-1]
            
        if oven_status[choose][0] == False:
            if oven_status[choose][1] == True:
                oven_stack[choose].append([time_stamp,pallet_no,duration])
                c = ovencounter[choose,1] 
                ovencounter[choose] = [indd,c+1]
                flag = 1
            elif oven_status[choose][1] == False:
                oven_stack[choose].append([time_stamp,pallet_no,duration])
                c = ovencounter[choose,1] 
                ovencounter[choose] = [indd,c+1]
                flag = 1
            if cooling_temp == 43:
                oven_status[choose][1]=False
            elif cooling_temp == 71:
                oven_status[choose][1]= True
        else:
            if ((old_time_stamp-datetime.datetime(2022,2,1)).total_seconds() + old_duration) <= (time_stamp-datetime.datetime(2022,2,1)).total_seconds():        
                oven_status[choose][0] = False 
                flag = 0
            else:
                busy = 1
        if flag == 1:
            ovens = np.delete(ovens, np.where(ovens == choose))
            break
  

    old_time_stamp = time_stamp        
    old_duration = duration

In [8]:
def to_visualize(oven_stack):
    
    results = []
    
    for oven_number in range(len(oven_stack)):
        default_time_stamp = oven_stack[oven_number][0][0]
        
        for i in range(len(oven_stack[oven_number])):
            time_stamp = oven_stack[oven_number][i][0]
            product_name = oven_stack[oven_number][i][1]
            duration = oven_stack[oven_number][i][2]
            start_time = (time_stamp-default_time_stamp).total_seconds()
            finish_time = (start_time + duration*60)
        
            results.append({'Job': product_name,
                 'Machine': oven_number+1,
                 'Start': start_time,
                 'Duration': duration,
                 'Finish': finish_time})
                                               
    return results

In [9]:
results= to_visualize(oven_stack)

df = pd.DataFrame(results, columns=['Job', 'Machine', 'Start', 'Duration', 'Finish'])
df=df.rename(columns={"Job": "Task"})
df.Start = pd.to_datetime(df.Start, unit='s', errors='coerce')
df.Finish = pd.to_datetime(df.Finish, unit='s', errors='coerce')
df.Duration = df.Duration * 60
df = df[['Machine','Start','Duration', 'Finish','Task']]
df['Machine'] = 'O' + df['Machine'].astype(str)
#print(df)

colors = {1: 'rgb(220, 0, 0)',
          2: (1, 0.9, 0.16),
          3: 'rgb(0, 255, 100)',
          4: 'rgb(220, 0, 0)',
          5: (1, 0.9, 0.16),
          6: 'rgb(0, 255, 100)',}

fig = ff.create_gantt(df, group_tasks=True, show_colorbar=True)
fig.show()

In [10]:
fig = px.timeline(df, x_start='Start', x_end='Finish', y='Machine', color='Machine')
fig.show()

In [11]:
import sqlite3
conn = sqlite3.connect('oven_scheduledatabase')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS products (Machine, Start, Duration, Finish, Task)')
conn.commit()


df.to_sql('products', conn, if_exists='replace', index = False)
c.execute('''  
SELECT * FROM products
          ''')


for row in c.fetchall():
    print (row)






('O1', '1970-01-01 00:00:00', 27000, '1970-01-01 07:30:00', 'A3')
('O1', '1970-01-01 12:05:46', 18000, '1970-01-01 17:05:46', 'B3')
('O1', '1970-01-02 21:53:53', 27000, '1970-01-03 05:23:53', 'A2')
('O1', '1970-01-03 04:44:15', 27000, '1970-01-03 12:14:15', 'B6')
('O1', '1970-01-03 09:03:04', 27000, '1970-01-03 16:33:04', 'A2')
('O1', '1970-01-04 04:41:44', 27000, '1970-01-04 12:11:44', 'A3')
('O1', '1970-01-04 07:53:44', 27000, '1970-01-04 15:23:44', 'B6')
('O1', '1970-01-04 13:55:40', 14400, '1970-01-04 17:55:40', 'C0')
('O1', '1970-01-05 12:31:36', 27600, '1970-01-05 20:11:36', 'A6')
('O1', '1970-01-05 19:21:33', 27000, '1970-01-06 02:51:33', 'A3')
('O1', '1970-01-06 00:08:02', 27600, '1970-01-06 07:48:02', 'A4')
('O1', '1970-01-06 00:13:34', 27000, '1970-01-06 07:43:34', 'A3')
('O1', '1970-01-06 15:17:37', 27000, '1970-01-06 22:47:37', 'A2')
('O1', '1970-01-06 21:59:59', 18000, '1970-01-07 02:59:59', 'B3')
('O1', '1970-01-07 05:16:00', 27600, '1970-01-07 12:56:00', 'A4')
('O1', '19